In [ ]:
import cv2
import numpy as np

def anms_keypoints(image, keypoints, max_keypoints):
    # Convert the image to grayscale if it's a color image
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    # Compute the response values for each keypoint
    responses = np.array([gray[y, x] for x, y in keypoints])

    # Initialize the suppression radius for each keypoint
    suppression_radius = np.full_like(responses, np.inf)

    # Iterate through each keypoint and find its suppression radius
    for i in range(len(keypoints)):
        for j in range(len(keypoints)):
            if responses[j] > responses[i]:
                # Compute the Euclidean distance between keypoints i and j
                distance = np.sqrt((keypoints[j][0] - keypoints[i][0])**2 + (keypoints[j][1] - keypoints[i][1])**2)

                # Update the suppression radius if the distance is smaller
                suppression_radius[i] = min(suppression_radius[i], distance)

    # Sort keypoints based on responses in descending order
    sorted_indices = np.argsort(responses)[::-1]

    # Initialize the final selected keypoints
    selected_keypoints = []

    # Keep adding keypoints to the final list until the desired number is reached
    for i in range(min(max_keypoints, len(keypoints))):
        # Get the current keypoint
        current_keypoint = keypoints[sorted_indices[i]]

        # Check if the current keypoint is not too close to any previously selected keypoints
        if all(np.sqrt((current_keypoint[0] - kp[0])**2 + (current_keypoint[1] - kp[1])**2) > suppression_radius[sorted_indices[i]] for kp in selected_keypoints):
            selected_keypoints.append(current_keypoint)

    return np.array(selected_keypoints)

# Example usage
image = cv2.imread("your_image.jpg")
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Use any keypoint detector (e.g., FAST) to obtain initial keypoints
keypoints = cv2.FastFeatureDetector_create().detect(gray_image)

# Apply ANMS to filter the keypoints
max_keypoints = 500
selected_keypoints = anms_keypoints(gray_image, [(int(kp.pt[0]), int(kp.pt[1])) for kp in keypoints], max_keypoints)

# Draw the selected keypoints on the image
output_image = image.copy()
for kp in selected_keypoints:
    cv2.circle(output_image, tuple(kp), 5, (0, 255, 0), -1)

cv2.imshow("ANMS Keypoints", output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

904435.0750059398